In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected = True)

pd.set_option('display.max_columns',None)

In [ ]:
stock_df = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")
all_stocks = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/stock_list.csv')

#Converting date column into datetime 
stock_df['Date'] = pd.to_datetime(stock_df['Date'])

Check for the information about each column

In [ ]:
stock_df.info()

1. Open, Close, High, Low have same value of Null values, which shows their might be some relation in that which we will examine later.

2. Null value in ExpectedDividend implies that no dividend was provided. 

3. RowId is combination of Date and SecuritiesCode

4. Date column is in object type which can be converted to datetime as per the usecase

In [ ]:
#Checking the total Null values
print("Total null values present in the DF is: ",stock_df.isnull().sum().sum())
print("="*50)

null_rec = []
for i in stock_df.columns:
    if stock_df[i].isnull().sum()>0:
        print("Null value in {} column is {}".format(i,stock_df[i].isnull().sum()))
    else:
        pass

    
print("="*50)
#Let us check the percentage of null values
for i in stock_df.columns:
    if stock_df[i].isnull().sum()>0:
        print("Null value in {} column is {}%".format(i,np.round(stock_df[i].isnull().sum()/len(stock_df)*100,2)))
    else:
        pass



In [ ]:
#We will look at the range of the data

print("Min date of the data is ", stock_df['Date'].min())
print("Max date of the data is ",stock_df['Date'].max())

In [ ]:
stock_df[['Open','Close','High','Low','Volume','ExpectedDividend','Target']].describe()

1. The opening price of stock ranges from 35 to 79030

2. The closing price of stocks ranges from 34 to 79080

3. Highest value of any stock is 79250.



The spread of all the columns is high

In [ ]:
#Let us plot some basic plots
for col in ['Open','Close','High','Low']:
    fig = px.histogram(x = stock_df[col]) 
    fig.update_xaxes(title=col)
    fig.update_yaxes(title = "Number of Rows")
    fig.update_layout(showlegend = True,
        title = {
            'text': 'Data Distribution ',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'} ,
            template="plotly_white")
    fig.show()


We can see that data in all the 4 colums are highly skewed, and follows almost same pattern.

In [ ]:
stock_category = all_stocks[['SecuritiesCode','33SectorName']]

stock_df_with_category =  pd.merge(stock_df,stock_category,how='left')
stock_df_with_category.head()

In [ ]:
stock_list = stock_df_with_category.groupby(['33SectorName']).size().reset_index(name= 'size')

fig = px.bar(x = stock_list["33SectorName"],
             y = stock_list['size'] , 
             color = stock_list["33SectorName"]) 

fig.update_xaxes(title="Categories of Stock")
fig.update_yaxes(title = "Number of Rows")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Data Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'} ,
        template="plotly_white")

fig.show()